<a href="https://colab.research.google.com/github/edgeofchaos42/ComplexityExplorer/blob/main/Session_6_Agentize_the_Landscape2_add_schedule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1- Review lesson 5 and intro

Script Notes: 

(Have all cells run prior to recording and map loaded) 

In lesson five we started to agentize the landscape, but creating sugar agents, who were added to our schedule. Remember if you shut down your colab kernel you will need to add the sugar-map.txt file again   

Now we want to place those sugar agents in time so eventually Traders can harvest and Trade them. 


Now a common convetion with google colab is to do a try and except statement with importing so if colab eve rmakes mesa a core library your code will not have to install. To okeep things neat we will add this to our import cell 

In [ ]:
#Delete this cell  
  !pip install mesa --quiet

In [2]:
try: 
  import mesa
except: 
  !pip install mesa --quiet
  import mesa
import numpy as np
import matplotlib.pyplot as plt



%matplotlib inline 

     |████████████████████████████████| 2.5 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 53.1 MB/s 
     |████████████████████████████████| 64 kB 2.4 MB/s 


In [7]:
class Sugar(mesa.Agent):
    """
    Sugar is a FSM that
    - contains an amount of sugar
    - grows 1 amount of sugar at each turn (Epstein's rule G1).
    """
    
    
    def __init__(self, unique_id, model, pos, max_sugar): #part 3
        super().__init__(unique_id, model) #part 3
        self.pos = pos
        self.amount = max_sugar
        self.max_sugar = max_sugar

--
**Part 3:**

We are going to build our spice just like our sugar. 



In [14]:
class Spice(mesa.Agent):
    """
    Spice is a FSM that
    - contains an amount of spice
    - grows 1 amount of spice at each turn.
    """

    def __init__(self, unique_id, model, pos, max_spice): #part 3
        super().__init__(unique_id, model) #part 3
        self.pos = pos
        self.amount = max_spice
        self.max_spice = max_spice
       

In [9]:
class Trader(mesa.Agent):
    """
    TraderAgent is a 
    - has a metabolism for sugar and spice
    - harvest and trades sugar and spice to survive and thrive
    """

    def __init__(self):
      print("I am a Trader")

## 2 Add Time
Now we need to add a scheduler to sequence agents. These are found in Mesas `time.py` file. mesa has 4 different activation regimes
1. RandomActivation, where each step the order of agents changes
2. SimultaneousActivation, where there is a step and advance functions are required. each agents process their step but does not apply them until advance
3. StagedAcitivation, where each step occurs in stages, for this model stage 1 may be sugar and spice grow, stage 2 agents move, stage 3 agents harvest, stage 4 agents trade-- repeat
4. RandomActivationByType in which different types of agents are activated

Although a few approaches could apply we will use RandomActivationbyType, as with previous examples all the code can be seen in [Mesa](https://github.com/projectmesa/mesa/blob/a2afbdb40a6f39e2781b30e0de6968ea3090fe38/mesa/time.py#L207). 

Later on  this choiuce will have some interesting implications if we are going to follow Growing Artificial Socieites exactly, but these show the power of codoing with python that there are numerous options to get the details one wants.  

But First
---
**Part 1**: We need to instnatiae our scheduler in this case `RandomActivationByType`. Using typical mesa convention we will call schedule...

---

**Part 2**: Next we can add the sugar agents in the for loop we will last lesson in the schedule. And check they are in the schedule by calling on their key


---
**Part 3:**
Now that this is done we do the same for spice and our landscape is created. First we can use the same loop and replciate the code near identically. We also need to update our spice agent and now we test and see our sugar and spice agents being created. 

WE have now completed lesson 6 and created our sugar and spice agents and place them on the mutlgrid. Next we will intiaite our traders. 

In [18]:
class SugarscapeG1mt(mesa.Model):

  def __init__(self, width=50, height=50): 
    
    self.width = width
    self.height = height

    #part 1: add schedule
    self.schedule = mesa.time.RandomActivationByType(self)
    self.grid = mesa.space.MultiGrid(self.width, self.height, torus=False)

    sugar_distribution = np.genfromtxt("sugar-map.txt") 
    spice_distribution = spice = np.flip(sugar_distribution,1)
    
    unique_id = 0
    for _, x, y in self.grid.coord_iter(): 
      max_sugar = sugar_distribution[x,y]
      if max_sugar > 0: 
        sugar = Sugar(unique_id, self, (x,y),max_sugar)
        self.grid.place_agent(sugar, (x, y))
        #Place Agent
        self.schedule.add(sugar) 
        #check
        print(self.schedule.agents_by_type[Sugar][unique_id]) #change to hidden attribute
        unique_id += 1

      #Part 3
      max_spice = spice_distribution[x,y]
      if max_spice > 0: 
        spice = Spice(unique_id, self, (x,y), max_spice)
        self.grid.place_agent(spice,(x,y))
        self.schedule.add(spice)
        print(self.schedule.agents_by_type[Spice][unique_id])
        unique_id += 1
         
    
        

Finally we will create a run cell that will initaite our models or models. 

In [ ]:
model = SugarscapeG1mt()